In [1]:
import numpy as np
from tqdm import tqdm
import wandb as wb
import imageio
from io import BytesIO
import pickle
import zarr

from src.codecs.imagecodecs import Jpeg2k, register_codecs

In [2]:
register_codecs()

In [3]:
def create_gif_or_mp4(np_images, filename, fps=10):
    # duration = 1000 / fps
    with imageio.get_writer(filename, fps=fps) as writer:
        for img in tqdm(np_images):
            writer.append_data(img)
    print(f"File saved as {filename}")

In [4]:
old_zarr = zarr.open(
    "/data/scratch/ankile/furniture-data-old/data/processed/sim/image_small/one_leg/data.zarr",
    mode="r",
)

In [5]:
old_zarr["color_image1"][0].shape

(224, 224, 3)

In [11]:
old_zarr["color_image1"].shape

(370234, 224, 224, 3)

In [6]:
image_compressor = Jpeg2k(level=75)

In [7]:
output_store = zarr.open(
    "/data/scratch/ankile/furniture-data-old/data/processed/sim/image_small/one_leg/data_compressed.zarr",
    mode="w",
)

In [8]:
_ = output_store.require_dataset(
    name="color_image1",
    shape=(10_000,) + old_zarr["color_image1"].shape[1:],
    chunks=(224, 224, 3),
    compressor=image_compressor,
    dtype=np.uint8,
)

# for i in tqdm(range(len(old_zarr["color_image1"]))):
output_store["color_image1"][:] = old_zarr["color_image1"][:10_000]

100%|██████████| 10000/10000 [02:47<00:00, 59.56it/s]


## Have a look at the compressed data to verify that it looks alright

In [9]:
store = zarr.open(
    "/data/scratch/ankile/furniture-data-old/data/processed/sim/image_small/one_leg/data_compressed.zarr",
)

In [16]:
(store["color_image1"].shape[0] - 224) / 10_000

224.0